# Idea
The main idea here is to create a new embedding for the words fed to the encoder. For this we train a summarizer transformer to learn the duden definitions of each word in our corpus. we then use the output of the encoder as our new embedding to train the sentiment analysis network on.